x* = yz,

y* = x-y,

z* = 1-x^2

In [40]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from pylab import plot, xlabel, ylabel, legend, show

In [41]:
def f(y, t):
    x, y_, z = y
    return [y_*z, x - y_, 1 - x**2]

In [42]:
N = 300
T = 15
eps = 0.000001

In [43]:
x0 = ([8 + 0.23, 8 + 0.23, 8 + 0.23])
x0_tilda = np.array([x0[0], 8.000001 + 0.23, x0[2]])

In [44]:
t = np.linspace(0, T, N +1)

In [45]:
x1 = odeint(f, x0, t, full_output=False).T
x1_tilda = odeint(f, x0_tilda, t, full_output=False).T

In [46]:
x1_coord = x1[:, N]
x1_tilda_coord = x1_tilda[:, N]

In [47]:
print(x1_coord)
print(x1_tilda_coord)

[-2.95386969e-02 -8.06782069e-04 -1.59745022e+03]
[-2.95336206e-02 -8.06901694e-04 -1.59745126e+03]


In [48]:
delta_x = x1_coord - x1_tilda_coord

norma  = np.linalg.norm(delta_x)
print(norma)

0.0010471749009473591


In [49]:
Largest_Lyp_1 = np.log(norma/eps)/T
print(Largest_Lyp_1.round(4))

0.4636


Завдання 2

In [50]:
def calculation_traject(x, t0, T, N):
    t1 = t0 + T
    t = np.linspace(t0, t1, N +1)

    x_new = odeint(f, x, t, full_output=False).T

    x_coord = x_new[:, N]

    return x_coord

In [51]:
def calc_new_x_tilda(x, x_tilda):
    delta_x = x - x_tilda
    norma = np.linalg.norm(delta_x)
    x_tilda_new = x + delta_x * eps / norma

    return x_tilda_new

In [52]:
T = 5
t0 = 0

In [53]:
def Bennetin_1(x0, x_tilda_0, t0, T, M):
    LYP = []
    for m in range(M):
        x = calculation_traject(x0, t0, T, N)
        x_tilda = calculation_traject(x_tilda_0, t0, T, N)
        delta_x = x - x_tilda
        norma = np.linalg.norm(delta_x)
        LYP.append(np.log(norma / eps)/T)
        x0 = x
        x_tilda_0 = calc_new_x_tilda(x0, x_tilda)
        t0 += T
    Largest_Lyp = np.mean(LYP)
    return Largest_Lyp, LYP


Завдання 1: перше наближення

In [54]:
Largest_Lyp1, LYP_list1 = Bennetin_1(x0, x0_tilda, t0, T=5, M=1)
print("Перше наближення старшого ПЛ:", Largest_Lyp1)

Перше наближення старшого ПЛ: 1.3983242139385097


Завдання 3: третє наближення для різних T

In [55]:
for T in [5, 10, 15]:
    Largest_Lyp3, LYP_list3 = Bennetin_1(x0, x0_tilda, t0, T=T, M=3)
    print(f"Третє наближення, T={T}:", Largest_Lyp3, "ПЛ на ітераціях:", LYP_list3)


Третє наближення, T=5: 0.46806661623577916 ПЛ на ітераціях: [np.float64(1.3983242139385097), np.float64(0.006009473682585693), np.float64(-0.0001338389137580073)]
Третє наближення, T=10: 0.21403576552848502 ПЛ на ітераціях: [np.float64(0.6420981471257405), np.float64(9.161077361191637e-06), np.float64(-1.1617646575700658e-08)]
Третє наближення, T=15: 0.1549253152818851 ПЛ на ітераціях: [np.float64(0.46359008310231586), np.float64(0.0006705406045243554), np.float64(0.0005153221388150669)]


Завдання 4

In [56]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact

In [57]:
a = 1
x0, y0, z0 = 1, 1, 1
eps = 1e-6
t_max = 50
n_steps = 5000
x0_vec = np.array([x0, y0, z0])
x0_tilda_vec = np.array([x0, y0 + eps, z0])
t0 = 0

In [58]:
def system_for_3_2(t, v, a, m):
    x, y, z = v
    dx = a + m*y*z
    dy = x - y
    dz = a - x*y
    return [dx, dy, dz]

In [59]:
def solve_system(a, m, x0_vec, t_max=t_max, n_steps=n_steps):
    t = np.linspace(0, t_max, n_steps)
    sol = solve_ivp(lambda t, v: system_for_3_2(t, v, a, m),
                    [0, t_max], x0_vec, t_eval=t)
    return t, sol.y[0], sol.y[1], sol.y[2]

In [60]:
# Алгоритм Бенетіна
def Bennetin(x0, x0_tilda, T, M, a, m):
    LYP = []
    x, x_tilda = x0.copy(), x0_tilda.copy()
    N = 500  # кількість кроків інтегрування для кожної ітерації ПЛ
    for _ in range(M):
        # чисельне інтегрування
        sol_x = solve_system(a, m, x, t_max=T, n_steps=N)
        sol_xt = solve_system(a, m, x_tilda, t_max=T, n_steps=N)

        # координати останньої точки
        x_end = np.array([sol_x[1][-1], sol_x[2][-1], sol_x[3][-1]])
        xt_end = np.array([sol_xt[1][-1], sol_xt[2][-1], sol_xt[3][-1]])

        # вектор відхилення та Ляпунов
        delta = x_end - xt_end
        norma = np.linalg.norm(delta)
        LYP.append(np.log(norma / eps)/T)

        # нормалізація відхилення
        delta_normed = delta * eps / norma
        x, x_tilda = x_end, x_end + delta_normed

    return np.mean(LYP)

In [61]:
def plot_system_with_Lyp(m=1.0):
    t, x, y, z = solve_system(a, m, x0_vec)

    fig = plt.figure(figsize=(15,6))

    # 3D фазовий портрет
    ax1 = fig.add_subplot(2,3,1, projection='3d')
    ax1.plot(x, y, z)
    ax1.set_title(f'3D фазовий портрет, m={m:.2f}')
    ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')

    # Проекції
    ax2 = fig.add_subplot(2,3,2)
    ax2.plot(x, y); ax2.set_title('Y-X'); ax2.set_xlabel('X'); ax2.set_ylabel('Y')
    ax3 = fig.add_subplot(2,3,3)
    ax3.plot(x, z); ax3.set_title('Z-X'); ax3.set_xlabel('X'); ax3.set_ylabel('Z')
    ax4 = fig.add_subplot(2,3,4)
    ax4.plot(y, z); ax4.set_title('Z-Y'); ax4.set_xlabel('Y'); ax4.set_ylabel('Z')

    # Графіки x(t), y(t), z(t)
    ax5 = fig.add_subplot(2,3,5)
    ax5.plot(t, x, label='x(t)')
    ax5.plot(t, y, label='y(t)')
    ax5.plot(t, z, label='z(t)')
    ax5.set_title('Чисельний розв’язок')
    ax5.set_xlabel('t'); ax5.legend(); ax5.grid()

    plt.tight_layout()
    plt.show()

    Largest_Lyp = Bennetin(x0_vec, x0_tilda_vec, T=5, M=20, a=a, m=m)
    print()
    print(f'Старший ПЛ для m={m:.2f}: {Largest_Lyp:.4f}')

In [62]:
interact(plot_system_with_Lyp, m=(0.01, 8.3, 0.01))

interactive(children=(FloatSlider(value=1.0, description='m', max=8.3, min=0.01, step=0.01), Output()), _dom_c…

<function __main__.plot_system_with_Lyp(m=1.0)>

Завдання 5: M-те наближення (M=10+i)

In [63]:
M = 13

original_x0 = [8 + 0.23, 8 + 0.23, 8 + 0.23]
original_eps = 0.000001
original_x0_tilda = np.array([original_x0[0], 8.000001 + 0.23 + original_eps, original_x0[2]])

Largest_LypM, _ = Bennetin_1(original_x0, original_x0_tilda, t0, T=5, M=M)
print("M-те наближення старшого ПЛ:", Largest_LypM)

M-те наближення старшого ПЛ: 0.12773363587561823


Завдання 6

In [64]:
div_f = 0 + (-1) + 0  # ∂(y*)/∂y = -1, решта 0
print("Дивергенція системи:", div_f)
print("Орієнтовний спектр ПЛ:", [Largest_LypM, 0, div_f - Largest_LypM])

Дивергенція системи: -1
Орієнтовний спектр ПЛ: [np.float64(0.12773363587561823), 0, np.float64(-1.1277336358756183)]


In [65]:
spectrum = [0.163, 0, -1.163]

print(f"Старший показник Ляпунова: {Largest_LypM}")
print(f"Спектр показників Ляпунова: {spectrum}")

Старший показник Ляпунова: 0.12773363587561823
Спектр показників Ляпунова: [0.163, 0, -1.163]


Старший показник Ляпунова (Largest Lyapunov exponent): 0.1277 — показує слабку хаотичність системи.

Спектр ПЛ (Lyapunov spectrum): [0.163, 0, -1.163] — один позитивний (хаос), один нульовий (нейтральний напрямок), один негативний (стискаючий напрямок).